In [ ]:
from yargy import ( Parser, rule, or_, and_, not_ )
from yargy.predicates import ( true, gram, eq, in_, type, normalized, dictionary, gte, lte, AndPredicate )
from yargy.pipelines import ( pipeline, morph_pipeline )
from yargy.interpretation import ( fact, attribute )
from yargy.relations import gnc_relation, number_relation, main
from yargy.predicates.constructors import ParameterPredicateScheme, ParameterPredicate
from yargy.predicates.bank import morph_required


def AndPredicate_constrain(self, token):
    if not hasattr(token, 'constrained'):
        return token

    forms = None

    for p in self.predicates:
        if p(token):
            constrained = p.constrain(token)

            if not hasattr(constrained, 'forms'):
                continue

            if not forms:
                forms = set(p.constrain(token).forms)
            else:
                forms &= set(p.constrain(token).forms)

    return token.constrained(list(forms))

AndPredicate.constrain = AndPredicate_constrain


class grams(ParameterPredicateScheme):
    def activate(self, context):
        for gr in self.value:
            context.tokenizer.morph.check_gram(gr)
        return GramsPredicate(self.value)


class GramsPredicate(ParameterPredicate):
    @morph_required
    def __call__(self, token):
        return any(
            all((v in _.grams for v in self.value))
            for _ in token.forms
        )

    def constrain(self, token):
        return token.constrained([
            _ for _ in token.forms
            if all((v in _.grams for v in self.value))
        ])

    @property
    def label(self):
        return "gram('%s')" % self.value


class not_grams(ParameterPredicateScheme):
    def activate(self, context):
        for gr in self.value:
            context.tokenizer.morph.check_gram(gr)
        return NotGramsPredicate(self.value)


class NotGramsPredicate(ParameterPredicate):
    @morph_required
    def __call__(self, token):
        return any(
            True
            for _ in token.forms
            if not all((v in _.grams for v in self.value))
        )

    def constrain(self, token):
        return token.constrained([
            _ for _ in token.forms
            if not all((v in _.grams for v in self.value))
        ])

    @property
    def label(self):
        return "gram('%s')" % self.value


None
None
None
красный молочай
красный молочай
красный молочай
красный молочай
красный молочай
None
None
красный молочай
None
None
None
None
None
None
красный молочай
красный молочай
красный молочай
None
None
None
None
красный молочай
красный молочай
красный молочай
красный молочай
красный молочай
красный молочай
None
красный молочай
красный молочай


In [ ]:
# https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html
# http://opencorpora.org/dict.php?act=gram

from db import db
from typing import Optional
import pymorphy2


morph = pymorphy2.MorphAnalyzer()

Collocation = fact(
    'Collocation',
    [attribute('main').repeatable(), attribute('dependent').repeatable()]
)

Entity = fact(
    'Entity',
    ['subject', 'predicate', 'prep']
)

# NOUN = or_(
#     and_(
#         gram("NOUN"),
#         not_(gram("PREP")), # Предлоги
#         not_(gram("PRCL")), # Частицы
#         not_(eq('алиса')),
#     ),
#     # больной проказой
#     and_(
#         gram("ADJF"),
#         gram("Subx"),
#     ),
#     # радиоведущий
#     # телевизионный ведущий
#     # заключённый, прокаженный
#     and_(
#         gram("PRTF"),
#         gram("Subx"),
#     ),
#     eq('хрен'), # Является частицей
#     eq('уж'), # Является частицей
#     normalized('ток'), # Является частицей, видимо
#     normalized('тип'), # Является частицей, видимо
# )

# ADJF = or_(
#     and_(
#         gram("ADJF"),
#         not_(gram("Apro")), # такой
#         not_(dictionary({'съедобное', 'полезные'})),
#         # Захватывает Допустим как Допустимый
#         not_(gram("VERB")),
#     ),
#     # орбитальная:ADJF пилотируемая:PRTF станция
#     and_(
#         gram("PRTF"),
#         not_(gram("Subx")),
#     )
# )

# PREP = gram("PREP")
# CONJ = gram("CONJ")

# кузнецкий (и) угольный бассейн
# DEPENDENT = rule(
#     rule(
#         ADJF.interpretation(
#             Collocation.dependent
#         ),
#         CONJ.optional()
#     ).optional(),
#     ADJF.interpretation(
#         Collocation.dependent
#     ),
# )

# # ягода малинка
# MAIN = rule(
#     NOUN.interpretation(
#         Collocation.main
#     ),
#     eq('-').optional(),
#     NOUN.optional().interpretation(
#         Collocation.main
#     )
# )

# COLLOCATION = or_(
#     MAIN,
#     # сладкая и сочная ягода малинка поваренная пищевая
#     # курица, курицу
#     # зубная паста
#     rule(DEPENDENT.optional(), MAIN),
#     # орехи грецкие
#     rule(MAIN, DEPENDENT.optional(),
#     ),
# ).interpretation(
#     Collocation
# )

# ENTITY = or_(
#     COLLOCATION.interpretation(
#         Entity.subject
#     ),
#     # обвиняемый по делу
#     # шампунь для волос
#     # бутерброды с кока колой
#     # мороженое в виде какашки
#     # артемовск в донецкой области, партия правого крыла
#     # служащий сферы обслуживания
#     # диски сложенные стопочкой predicate
#     rule(
#         COLLOCATION.interpretation(
#             Entity.subject
#         ),
#         rule(
#             PREP.optional().interpretation(
#                 Entity.prep
#             ),
#             COLLOCATION.interpretation(
#                 Entity.predicate
#             )
#         ),
#     ),
# ).interpretation(
#     Entity
# )


def extract_entity(command: str) -> Optional[str]:
    matches = list(parser.findall(command))
    
    if not matches:
        return None

    match = max(matches, key=lambda m: len(m.tokens))

    result = []
    return
    e = match.fact

    print(e)

    s = e.subject
    sm0 = morph.parse(s.main[0])[0]

    dependent_tags = {'nomn', sm0.tag.number}

    if sm0.tag.number == 'sing':
        dependent_tags.add(sm0.tag.gender)

    for dep in s.dependent:
        result.append(
            morph.parse(dep)[0].inflect(dependent_tags).word
        )

    if sm0.tag.animacy == 'anim':
        result.append(sm0.inflect({'nomn'}).word)
    else:
        result.append(sm0.word)

    result += s.main[1:]


    if e.predicate:
        if e.prep:
            result.append(e.prep)

        result += e.predicate.dependent
        result += e.predicate.main
    
    return " ".join(result)


# print(extract_entity("раковина беспозвоночного животного"))


for doc in db.eat_eatable.find({'is_hidden': {"$ne":True}}):
# for doc in db.eat_guess.find({"is_user": True}):
    command = doc['name']
    expected = doc['name']

    if '-' in command or '/' in command or '"' in command or '(' in command:
        continue

    matches = parser.findall(command)

    if not matches:
        print("❗️", command)
        continue

    entities = []
    found = False
    for match in matches:
        entity = command[match.span.start:match.span.stop]
        entities.append(entity)

        if entity == expected:
            found = True
            break

    if not found:
        print(" | ".join(entities), "!=", expected)

In [ ]:
from ipymarkup import show_span_ascii_markup as show_markup


for doc in db.eat_guess.find({"is_user": True}):
    command = doc['riddle']


    match = parser.find(command)

    if not match:
        print("❗️", command)
        continue

    show_markup(command, [match.span])